In [28]:
from numpy import dot
from numpy.linalg import norm
import statistics

In [29]:
import numpy as np
from numpy.linalg import norm
from numpy import dot

class kmeans:

    def __init__(self, n_clusters, max_iter=100):
        self.n_clusters = n_clusters
        self.max_iter = max_iter

        
    def jaccard_similarity(self,a,b):
        a=set(a)
        b=set(b)
        s1=a.intersection(b)
        s2=a.union(b)
        return len(s1)/len(s2)
    
    def cosine_similarity(self,v1,v2):
        return dot(v1, v2)/(norm(v1)*norm(v2))

    def initializ_centroids(self, X): # other initialization
        random_idx = np.random.permutation(X.shape[0])
        centroids = X[random_idx[:self.n_clusters]]
        return centroids

    def compute_centroids(self, X, labels):
        centroids = np.zeros((self.n_clusters, X.shape[1]))
        for k in range(self.n_clusters):
            centroids[k, :] = np.mean(X[labels == k, :], axis=0)
        return centroids
    
    def metadata(self,dfc,labels):
        cluster_num = self.n_clusters
        y= dfc[:,0]
        X = dfc[:,1:]  #preparing values in numpy format
        sum = np.zeros((cluster_num, X.shape[1]))
        sum_sq = np.zeros((cluster_num, X.shape[1]))
        N = np.zeros(cluster_num)
        #new_centroids = np.zeros((cluster_num, X.shape[1]))     #new centroids
        movieid = []
        for k in range(cluster_num):
            sum[k,:] = np.sum(X[labels==k,:],axis=0)
            sum_sq[k,:]=  np.sum((X[labels==k,:])**2,axis=0) 
            N[k] = X[labels==k].shape[0] 
            #new_centroids[k] = sum[k]/N[k]
            movieid.append(list(y[labels==k]))
        return (N,sum,sum_sq,movieid)
    


    def compute_distance(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            for j in range(X.shape[0]):
                distance[j][k]=self.cosine_similarity(X[j],centroids[k])
        #pring(distance.shape)
        return distance


    
    def centroids(self, X):
        self.centroids = self.initializ_centroids(X[:,1:])
        for i in range(self.max_iter):  #it would summarise any x input giveen to it
            old_centroids = self.centroids
            distance = self.compute_distance(X[:,1:], old_centroids)
            self.labels = np.argmax(distance,axis=1) # finding closest distance returns an array of row indices for which 
            self.centroids = self.compute_centroids(X[:,1:], self.labels) 
            if np.all(old_centroids == self.centroids):
                #print(self.centroids)
                return self.metadata(X,self.labels)
        return self.metadata(X,self.labels) 


In [30]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd



class bfr:

    def __init__(self, n_clusters,meta, max_iter=100):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.meta = meta
        self.centroids = self.compute_centroids(self.meta,self.n_clusters)

    
    def jaccard_similarity(self,a,b):
        a=set(a)
        b=set(b)
        s1=a.intersection(b)
        s2=a.union(b)
        return len(s1)/len(s2)
    
    def cosine_similarity(self,v1,v2):
        denom = norm(v1)*norm(v2)
        if denom == 0 :
            denom = 0.00000001
        return dot(v1, v2)/denom
    
    def compute_centroids(self, summary,cluster_num):  #takes in summed summary
        centroids = np.zeros((cluster_num, len(summary[1][1])))
        for k in range(cluster_num):
            centroids[k, :] = summary[1][k]/summary[0][k]
        return centroids
    

    
    def metadata(self,dfc,labels,cluster_num):
        y= dfc[:,0]
        X=dfc[:,1:]
        sum = np.zeros((cluster_num, X.shape[1]))
        sum_sq = np.zeros((cluster_num, X.shape[1]))
        N = np.zeros(cluster_num)
        #new_centroids = np.zeros((cluster_num, X.shape[1]))     #new centroids
        movieid = []
        for k in range(cluster_num):
            sum[k,:] = np.sum(X[labels==k,:],axis=0)   + self.meta[1][k]
            sum_sq[k,:]=  np.sum((X[labels==k,:])**2,axis=0) + self.meta[2][k]
            N[k] = X[labels==k].shape[0] + self.meta[0][k]
            #new_centroids[k] = sum[k]/N[k]
            movieid.append(list(y[labels==k]))
            #movieid[k].extend(self.meta[3][k])
        return (N,sum,sum_sq,movieid)
    


    def compute_distance(self, X, centroids,cluster_num):
        distance = np.zeros((X.shape[0], cluster_num))
        #print(len(centroids))
        for k in range(cluster_num):
            for j in range(X.shape[0]):
                distance[j][k]=self.cosine_similarity(X[j],centroids[k])  #centroid is the k column
        #pring(distance.shape)
        return distance


    
    def summarize(self, X):  #expects a centroid input
        distance = self.compute_distance(X[:,1:], self.centroids,self.n_clusters)
        self.labels = np.argmax(distance,axis=1) # finding closest distance returns an array of row indices for which 
        self.lab_dist = np.max(distance,axis=1)
        return self.metadata(X,self.labels,self.n_clusters) # take in dfd here and mordify metadata, dfd has all required information
            
    def active_centroid(self):
        return self.centroids
  

In [31]:
def distance_d3(n):
    file = open('movie_ratings.csv','r',buffering=3000000,encoding='utf-8')
    chunk_num=0
    results = ''
    results1=[[] for i in range(n)]
    while True:
        c=0
        tt=[]
        for line in file:
            tt.append(line.replace('\n','').split(','))
            c += 1
            if c==1000:
                break 
        tt=np.array(tt)
        tx=np.array(tt[:,1:],dtype='f')
        ind=np.arange(0,tt.shape[0])
        ind =ind.reshape([-1,1])
        ty=np.concatenate((ind,tx),axis=1)
        if chunk_num ==0 :
            kr=kmeans(n)
            results=kr.centroids(ty)
            chunk_num += 1
        else:
            ck = bfr(n,results)
            results = ck.summarize(ty) 
        for i in range(len(results[3])):
            for j in results[3][i]:
                results1[i].append(tt[int(j),0])
        if c<1000:
            break
    file.close()
    centroids1=[]
    for num,sums in zip(results[0],results[1]):
        centroids1.append(sums/num)
    df=pd.DataFrame(centroids1)
    cnt4_csv=df.to_csv('centroids_d1.csv',header=False,index=False)
    df4 = pd.DataFrame([])
    df4['Cluster id'] = np.arange(0,len(results1))
    df4['Movies in Cluster'] = results1
    df4_csv=df4.to_csv('clusters_d1.csv',header=True,index=False,sep='|')
    return

In [32]:
import numpy as np
from numpy.linalg import norm
from numpy import dot

class kmeansjc:

    def __init__(self, n_clusters, max_iter=100):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        
    def jaccard_similarity(self,a,b):
        a=set(a)
        b=set(b)
        s1=a.intersection(b)
        s2=a.union(b)
        return len(s1)/len(s2)
    
    def cosine_similarity(self,v1,v2):
        if len(v1)==0 or len(v2)==0:   #error proofing
            return 0
        return dot(v1, v2)/(norm(v1)*norm(v2))

    def initializ_centroids(self, X): #use d other initialization
        random_idx = np.random.permutation(len(X))
        centroids = X[random_idx[:self.n_clusters]]
        return centroids
    def initialize(self, X): #use d other initialization
        random_idx = np.random.permutation(X.shape[0])
        centroids = X[random_idx[:self.n_clusters]]
        return centroids


    
    def intracluster_distance(self,X):
        if len(X)==0:
            return 0
        sum = np.zeros(len(X))
        #print(sum)
        for i , val in enumerate(X):
            for j in X:
                sum[i] += self.jaccard_similarity(val,j)
        #print(sum)
        return np.max(sum)
    
    def compute_d(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            for j in range(X.shape[0]):
                #print((j,k))
                distance[j][k]=self.d4(X[j,:],centroids[k,:])
        #pring(distance.shape)
        return distance 
    
    def intracluster_d(self,X):
        if X.shape[0]==0:
            return 0
        sum = np.zeros(X.shape[0])
        #print(sum)
        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                sum[i] += self.d4(X[i,:],X[j,:])
        #print(sum)
        return np.max(sum)


    def compute_distance(self, X, centroids):
        distance = np.zeros((X.shape[0], self.n_clusters))
        for k in range(self.n_clusters):
            for j in range(X.shape[0]):
                distance[j][k]=self.jaccard_similarity(X[j],centroids[k])
        #pring(distance.shape)
        return distance
    
    def d1(self,x,y): #jaccard similarity based on genres of movies
        return self.jaccard_similarity(x[1],y[1])
    def d2 (self,x,y): #jaccard similarity based on tags of movies
        return self.jaccard_similarity(x[2],y[2])
    def d3 (self,x,y): #cosine similarity based on the ratings of the movies
        #print(x[3:])
        a = np.array(x[3:],dtype='f')
        b = np.array(y[3:],dtype='f')
        return self.cosine_similarity(a,b)
    def d4 (self,x,y):
        return 0.3*self.d1(x,y) + 0.25*self.d2(x,y) + 0.45*self.d3(x,y)

    def stable(self,X,n):
        for k in range(self.n_clusters):
            if n==0:  
                c=self.intracluster_distance(X[k==self.labels])
            else:
                c= self.intracluster_d(X[k==self.labels,:])
            if np.sum(self.dist[k==self.labels]) < c: # centroids should have greatest similarity
                return False
        return True
    
    def centroids(self, X, n=0):
        if n==0:
            self.centroids = self.initializ_centroids(X[:,1])
        else:
            self.centroids = self.initialize(X[:,1:])
        #print(X[:,1:].shape)
        for i in range(self.max_iter):  #it would summarise any x input giveen to it
            if n==0:
                distance = self.compute_distance(X[:,1], self.centroids)
            else:
                distance = self.compute_d(X[:,1:], self.centroids)
            self.labels = np.argmax(distance,axis=1) # finding closest distance returns an array of row indices for which 
            self.dist = np.max(distance,axis=1)
            if n==0:
                if self.stable(X[:,1],n):
                    return self.centroids
            else:
                #if self.stable(X[:,1:],n):
                return self.centroids
        return self.centroids 
    
    def predict(self, X,n=0):
        if n==0:
            distance = self.compute_distance(X[:,1], self.centroids)
        else:
            distance = self.compute_d(X[:,1:], self.centroids)
        return np.argmax(distance,axis=1)   #returns an array with indices coincidx to the movies ie return[0]=cluster of movie 0

In [33]:
##clustering based on genre
def distance_d1(n):
    file2 = open('movie_genre.csv','r',buffering=10000,encoding='UTF-8')
    chunk_num2=0
    results2 = ''
    cluster2 = [[] for i in range(n)]  #create another function so you don't have to put a hard number
    while True:
        c=0
        tt=[]
        for line in file2:
            tt.append(line.replace('\n','').split(','))
            c += 1
            if c==1000:
                break 
        tt=np.array(tt)
        if chunk_num2 == 0 :
            krjc=kmeansjc(n)
            results2=krjc.centroids(tt)
            chunk_num2 += 1

        cluster_array = krjc.predict(tt)
        for i,r in enumerate(cluster_array):
            cluster2[int(r)].append(tt[i][0])
        if c < 1000:
            break   
    file2.close()       
    df3 = pd.DataFrame([])
    df3['Cluster id'] = np.arange(0,len(cluster2))
    df3['Movies in Cluster'] = cluster2
    df3_csv=df3.to_csv('clusters_d2.csv',header=True,index=False,sep='|')
    centroids3 = pd.DataFrame([])
    centroids3['Cluster id'] = np.arange(0,len(results2))
    centroids3['Centroids'] = results2
    cnt3_csv=centroids3.to_csv('centroids_d2.csv',header=False,index=False,sep='|')
    return 

In [34]:
##clustering based on genre
def distance_d2(n):
    file3 = open('movie_tags.csv','r',buffering=200000,encoding='UTF-8')
    chunk_num3=0
    results3 = ''
    cluster3 = [[] for i in range(n)]  #create another function so you don't have to put a hard number
    while True:
        c=0
        tt=[]
        for line in file3:
            tt.append(line.replace('\n','').split(','))
            c += 1
            if c==100:
                break 
        tt=np.array(tt)
        if chunk_num3 == 0 :
            krjc3=kmeansjc(n)
            results3=krjc3.centroids(tt)
            chunk_num3 += 1

        cluster_array = krjc3.predict(tt)
        for i,r in enumerate(cluster_array):
            cluster3[int(r)].append(tt[i][0])
        if c < 100:
            break   
    file3.close()      
    df2 = pd.DataFrame([])
    df2['Cluster id'] = np.arange(0,len(cluster3))
    df2['Movies in Cluster'] = cluster3
    df2_csv=df2.to_csv('clusters_d3.csv',header=True,index=False,sep='|')
    centroids2 = pd.DataFrame([])
    centroids2['Cluster id'] = np.arange(0,len(results3))
    centroids2['Centroids'] = results3
    cnt2_csv=centroids2.to_csv('centroids_d3.csv',header=False,index=False,sep='|')
    return 

In [35]:
##clustering based on genre
import numpy as np
def distance_d4(n):
    file4 = open('combine.csv','r',buffering=3000000,encoding='UTF-8')
    chunk_num4=0
    results4 = ''
    cluster4 = [[] for i in range(n)]  #create another function so you don't have to put a hard number
    while True:
        c=0
        tt=[]
        for line in file4:
            tt.append(line.replace('\n','').split(','))
            c += 1
            if c==100:
                break 
        tt=np.array(tt)
        if chunk_num4 == 0 :
            krjc4=kmeansjc(n,max_iter=1)
            results4=krjc4.centroids(tt,1)
            chunk_num4 += 1

        cluster_array = krjc4.predict(tt,1)
        for i,r in enumerate(cluster_array):
            cluster4[int(r)].append(tt[i][0])
        if c < 100:
            break    #no need to return cluster 2 here since it is a global variable

    file4.close()        #clusters ( movieids) are in results[3]
    import pandas as pd
    df1 = pd.DataFrame([])
    df1['Cluster id'] = np.arange(0,len(cluster4))
    df1['Movies in Cluster'] = cluster4
    df1_csv=df1.to_csv('clusters_d4.csv',header=True,index=False,sep='|')
    centroids1 = pd.DataFrame(results4)
    cnt1_csv=centroids1.to_csv('centroids_d4.csv',header=False,index=True,sep='|')
    return 

In [36]:
print('Movie Clustering Application')
def main():
    n=input('Please input the number of clusters k=')
    n=int(n)
    print('Supported distance metrics\n\
         d1: jaccard similarity based on the genres of the movies \n\
         d2: jaccard similarity based on the tags of the movies \n\
         d3: cosine similarity based on the ratings of the movies \n\
         d4 = 0.3*d1 + 0.25*d2 + 0.45*d3')
    d=input('Please input selection from above (eg d1):')
    if d=='d1':
        print('Running...Please Wait')
        distance_d1(n)
    elif d=='d2':
        print('Running...Please Wait')
        distance_d2(n)
    elif d=='d3':
        print('Running...Please Wait')
        distance_d3(n)
    elif d=='d4':
        print('Running...Please Wait')
        distance_d4(n)
    else:
        print('Wrong input. Please try again')
        main()
    print('Done..Please check output files')
main()

Movie Clustering Application
Please input the number of clusters k=12
Supported distance metrics
         d1: jaccard similarity based on the genres of the movies 
         d2: jaccard similarity based on the tags of the movies 
         d3: cosine similarity based on the ratings of the movies 
         d4 = 0.3*d1 + 0.25*d2 + 0.45*d3
Please input selection from above (eg d1):d4
Running...Please Wait
Done..Please check output files
